<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/milvus_rag_with_dynamiq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/milvus_rag_with_dynamiq.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>

# Getting Started with Dynamiq and Milvus



[Dynamiq](https://dynamiq.ai/) is a powerful Gen AI framework that streamlines the development of AI-powered applications. With robust support for retrieval-augmented generation (RAG) and large language model (LLM) agents, Dynamiq empowers developers to create intelligent, dynamic systems with ease and efficiency.

In this tutorial, we’ll explore how to seamlessly use Dynamiq with [Milvus](https://milvus.io/), a high-performance, open-source vector database purpose-built for RAG workflows. Milvus excels at efficient storage, indexing, and retrieval of vector embeddings, making it an indispensable component for AI systems that demand fast and precise contextual data access.

This step-by-step guide will cover two core RAG workflows:

- **Document Indexing Flow**: Learn how to process input files (e.g., PDFs), transform their content into vector embeddings, and store them in Milvus. Leveraging Milvus’s high-performance indexing capabilities ensures your data is ready for rapid retrieval.

- **Document Retrieval Flow**: Discover how to query Milvus for relevant document embeddings and use them to generate insightful, context-aware responses with Dynamiq’s LLM agents, creating a seamless AI-powered user experience.

By the end of this tutorial, you’ll gain a solid understanding of how Milvus and Dynamiq work together to build scalable, context-aware AI systems tailored to your needs.

## Preparation


### Download required libraries


In [1]:
! pip install dynamiq pymilvus


> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

### Configure the LLM agent

We will use OpenAI as the LLM in this example. You should prepare the [api key](https://platform.openai.com/docs/quickstart) `OPENAI_API_KEY` as an environment variable.


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-***********"

## RAG - Document Indexing Flow

This tutorial demonstrates a Retrieval-Augmented Generation (RAG) workflow for indexing documents with Milvus as the vector database. The workflow takes input PDF files, processes them into smaller chunks, generates vector embeddings using OpenAI's embedding model, and stores the embeddings in a Milvus collection for efficient retrieval.

By the end of this workflow, you will have a scalable and efficient document indexing system that supports future RAG tasks like semantic search and question answering.

### Import Required Libraries and Initialize Workflow

In [3]:
# Importing necessary libraries for the workflow
from io import BytesIO
from dynamiq import Workflow
from dynamiq.nodes import InputTransformer
from dynamiq.connections import OpenAI as OpenAIConnection, Milvus as MilvusConnection, MilvusDeploymentType
from dynamiq.nodes.converters import PyPDFConverter
from dynamiq.nodes.splitters.document import DocumentSplitter
from dynamiq.nodes.embedders import OpenAIDocumentEmbedder
from dynamiq.nodes.writers import MilvusDocumentWriter

# Initialize the workflow
rag_wf = Workflow()


### Define PDF Converter Node

In [4]:
converter = PyPDFConverter(document_creation_mode="one-doc-per-page")
converter_added = rag_wf.flow.add_nodes(converter)  # Add node to the DAG (Directed Acyclic Graph)


### Define Document Splitter Node

In [5]:
document_splitter = DocumentSplitter(
    split_by="sentence",  # Splits documents into sentences
    split_length=10,      
    split_overlap=1,      
    input_transformer=InputTransformer(
        selector={
            "documents": f"${[converter.id]}.output.documents", 
        },
    ),
).depends_on(converter)  # Set dependency on the PDF converter
splitter_added = rag_wf.flow.add_nodes(document_splitter)  # Add to the DAG


### Define Embedding Node

In [8]:
embedder = OpenAIDocumentEmbedder(
    connection=OpenAIConnection(api_key=os.environ["OPENAI_API_KEY"]),  
    input_transformer=InputTransformer(
        selector={
            "documents": f"${[document_splitter.id]}.output.documents",  
        },
    ),
).depends_on(document_splitter)  # Set dependency on the splitter
document_embedder_added = rag_wf.flow.add_nodes(embedder)  # Add to the DAG


### Define Milvus Vector Store Node

In [9]:
vector_store = (
    MilvusDocumentWriter(
        connection=MilvusConnection(deployment_type=MilvusDeploymentType.FILE, uri="./milvus.db"),
        index_name="my_milvus_collection",  
        dimension=1536,  
        create_if_not_exist=True, 
        metric_type="COSINE"  
    )
    .inputs(documents=embedder.outputs.documents)  # Connect to embedder output
    .depends_on(embedder)  # Set dependency on the embedder
)
milvus_writer_added = rag_wf.flow.add_nodes(vector_store)  # Add to the DAG


2024-11-19 22:04:20 - WARNING - Environment variable 'MILVUS_API_TOKEN' not found
2024-11-19 22:04:20 - INFO - Pass in the local path ./milvus.db, and run it using milvus-lite
2024-11-19 22:04:20 - DEBUG - Created new connection using: 6f60987938684f109995723a3579d247
2024-11-19 22:04:20 - INFO - Collection my_milvus_collection already exists. Skipping creation.


> # Milvus Deployment Types
>
> Milvus offers two deployment types, catering to different use cases:
>
>
> ## 1. **MilvusDeploymentType.FILE**
>
> - Ideal for **local prototyping** or **small-scale data** storage.
> - Set the `uri` to a local file path (e.g., `./milvus.db`) to leverage [Milvus Lite](https://milvus.io/docs/milvus_lite.md), which automatically stores all data in the specified file.
> - This is a convenient option for **quick setup** and **experimentation**.
>
>
> ## 2. **MilvusDeploymentType.HOST**
>
> - Designed for **large-scale data** scenarios, such as managing over a million vectors.
>
> ### **Self-Hosted Server**
> - Deploy a high-performance Milvus server using [Docker or Kubernetes](https://milvus.io/docs/quickstart.md).
> - Configure the server’s address and port as the `uri` (e.g., `http://localhost:19530`).
> - If authentication is enabled:
>   - Provide `<your_username>:<your_password>` as the `token`.
> - If authentication is disabled:
>   - Leave the `token` unset.
>
> ### **Zilliz Cloud (Managed Service)**
> - For a fully managed, cloud-based Milvus experience, use [Zilliz Cloud](https://zilliz.com/cloud).
> - Set the `uri` and `token` according to the [Public Endpoint and API key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#cluster-details) provided in the Zilliz Cloud console.

### Define Input Data and Run the Workflow

In [11]:
file_paths = ["./pdf_files/WhatisMilvus.pdf"]
input_data = {
    "files": [
        BytesIO(open(path, "rb").read()) for path in file_paths 
    ],
    "metadata": [
        {"filename": path} for path in file_paths  
    ],
}

# Run the workflow with the prepared input data
inserted_data = rag_wf.run(input_data=input_data)


/var/folders/09/d0hx80nj35sb5hxb5cpc1q180000gn/T/ipykernel_31159/3145804345.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='./pdf_files/WhatisMilvus.pdf'>
  BytesIO(open(path, "rb").read()) for path in file_paths
2024-11-19 22:04:47 - INFO - Workflow a85f89a9-3dc3-4f00-8cef-9e415146a085: execution started.
2024-11-19 22:04:47 - INFO - Flow 4a40047b-6fd4-4fd8-9131-b90be506a1c6: execution started.
2024-11-19 22:04:47 - INFO - Node PyPDF File Converter - f2f04013-a27f-4adb-89f7-7a10e269ff8a: execution started.
2024-11-19 22:04:47 - INFO - Node PyPDF File Converter - f2f04013-a27f-4adb-89f7-7a10e269ff8a: execution succeeded in 59ms.
2024-11-19 22:04:47 - INFO - Node DocumentSplitter - fd0fe4a4-490a-4963-b712-848c7b6b05b8: execution started.
2024-11-19 22:04:47 - INFO - Node DocumentSplitter - fd0fe4a4-490a-4963-b712-848c7b6b05b8: execution succeeded in 6ms.
2024-11-19 22:04:47 - INFO - Node OpenAIDocumentEmbedder - a718a942-0780-463c-8b54-5cea9288bbbd: execution started.
202

Through this workflow, we have successfully implemented a document indexing pipeline using Milvus as the vector database and OpenAI's embedding model for semantic representation. This setup enables fast and accurate vector-based retrieval, forming the foundation for RAG workflows like semantic search, document retrieval, and contextual AI-driven interactions.

With Milvus's scalable storage capabilities and Dynamiq's orchestration, this solution is ready for both prototyping and large-scale production deployments. You can now extend this pipeline to include additional tasks like retrieval-based question answering or AI-driven content generation.

## RAG Document Retrieval Flow

In this tutorial, we implement a Retrieval-Augmented Generation (RAG) document retrieval workflow. This workflow takes a user query, generates a vector embedding for it, retrieves the most relevant documents from a Milvus vector database, and uses a large language model (LLM) to generate a detailed and context-aware answer based on the retrieved documents.

By following this workflow, you will create an end-to-end solution for semantic search and question answering, combining the power of vector-based document retrieval with the capabilities of OpenAI’s advanced LLMs. This approach enables efficient and intelligent responses to user queries by leveraging the stored knowledge in your document database.

### Import Required Libraries and Initialize Workflow

In [12]:
from dynamiq import Workflow
from dynamiq.connections import OpenAI as OpenAIConnection, Milvus as MilvusConnection, MilvusDeploymentType
from dynamiq.nodes.embedders import OpenAITextEmbedder
from dynamiq.nodes.retrievers import MilvusDocumentRetriever
from dynamiq.nodes.llms import OpenAI
from dynamiq.prompts import Message, Prompt

# Initialize the workflow
retrieval_wf = Workflow()


### Define OpenAI Connection and Text Embedder

In [13]:
# Establish OpenAI connection
openai_connection = OpenAIConnection(api_key=os.environ["OPENAI_API_KEY"])

# Define the text embedder node
embedder = OpenAITextEmbedder(
    connection=openai_connection,
    model="text-embedding-3-small",  
)

# Add the embedder node to the workflow
embedder_added = retrieval_wf.flow.add_nodes(embedder)


### Define Milvus Document Retriever

In [14]:
document_retriever = (
    MilvusDocumentRetriever(
        connection=MilvusConnection(
            deployment_type=MilvusDeploymentType.FILE,
            uri="./milvus.db"  
        ),
        index_name="my_milvus_collection",  
        dimension=1536, 
        top_k=5, 
    )
    .inputs(embedding=embedder.outputs.embedding)  # Connect to embedder output
    .depends_on(embedder)  # Dependency on the embedder node
)

# Add the retriever node to the workflow
milvus_retriever_added = retrieval_wf.flow.add_nodes(document_retriever)


2024-11-19 22:04:55 - WARNING - Environment variable 'MILVUS_API_TOKEN' not found
2024-11-19 22:04:55 - INFO - Pass in the local path ./milvus.db, and run it using milvus-lite
2024-11-19 22:04:55 - DEBUG - Created new connection using: 24547e0ab0c84d858a935b9373c59e77
2024-11-19 22:04:55 - INFO - Collection my_milvus_collection already exists. Skipping creation.


###  Define the Prompt Template

In [15]:
# Define the prompt template for the LLM
prompt_template = """
Please answer the question based on the provided context.

Question: {{ query }}

Context:
{% for document in documents %}
- {{ document.content }}
{% endfor %}
"""

# Create the prompt object
prompt = Prompt(messages=[Message(content=prompt_template, role="user")])


### Define the Answer Generator


In [16]:
answer_generator = (
    OpenAI(
        connection=openai_connection,
        model="gpt-4o",  
        prompt=prompt, 
    )
    .inputs(
        documents=document_retriever.outputs.documents,  
        query=embedder.outputs.query,  
    )
    .depends_on([document_retriever, embedder])  # Dependencies on retriever and embedder
)

# Add the answer generator node to the workflow
answer_generator_added = retrieval_wf.flow.add_nodes(answer_generator)


### Run the Workflow

In [20]:
# Run the workflow with a sample query
sample_query = "What is the Advanced Search Algorithms in Milvus?"

result = retrieval_wf.run(input_data={"query": sample_query})

answer = result.output.get(answer_generator.id).get("output", {}).get("content")
print(answer)

2024-11-19 22:05:45 - INFO - Workflow cf4022d8-8576-441d-bd7e-9d94c2c79194: execution started.
2024-11-19 22:05:45 - INFO - Flow 4a40047b-6fd4-4fd8-9131-b90be506a1c6: execution started.
2024-11-19 22:05:45 - INFO - Node OpenAITextEmbedder - c8919acd-eb18-48b1-b9c6-f0e51375b804: execution started.
2024-11-19 22:05:45 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-19 22:05:45 - INFO - Node OpenAITextEmbedder - c8919acd-eb18-48b1-b9c6-f0e51375b804: execution succeeded in 339ms.
2024-11-19 22:05:45 - INFO - Node MilvusDocumentRetriever - 90168362-225b-4c78-b69e-38a582debb3c: execution started.
2024-11-19 22:05:45 - INFO - Node MilvusDocumentRetriever - 90168362-225b-4c78-b69e-38a582debb3c: execution succeeded in 6ms.
2024-11-19 22:05:45 - INFO - Node LLM - 2ca62c70-20ff-4875-8f89-66df656f91f6: execution started.
2024-11-19 22:05:48 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-19 22:05:48 - INFO 

The advanced search algorithms in Milvus include several types of searches designed to meet different use case demands. These are:

1. **ANN Search**: Finds the top K vectors closest to your query vector.
2. **Filtering Search**: Performs ANN search under specified filtering conditions.
3. **Range Search**: Finds vectors within a specified radius from your query vector.
4. **Hybrid Search**: Conducts ANN search based on multiple vector fields.
5. **Keyword Search**: Keyword search based on BM25.
6. **Reranking**: Adjusts the order of search results based on additional criteria or a secondary algorithm, refining the initial ANN search results.
7. **Fetch**: Retrieves data by their primary keys.
8. **Query**: Retrieves data using specific expressions.

These algorithms are part of Milvus's robust capabilities to handle complex search requirements efficiently.


This RAG document retrieval workflow effectively combines state-of-the-art technologies for query understanding, document retrieval, and natural language generation. By embedding queries, retrieving contextually relevant documents, and generating insightful answers, this workflow enables intelligent and context-aware interactions. This workflow can be extended to handle complex multi-document queries, knowledge-based systems, and intelligent assistants, making it a foundational building block for advanced AI applications.